## Neural Networks

In [1]:
import torch

In [2]:
from torch.autograd import Variable

In [3]:
import torch.nn as nn

In [4]:
import torch.nn.functional as F

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution kernel
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [6]:
net = Net()

In [7]:
print(net)

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


In [8]:
params = list(net.parameters())
print(len(params))

10


In [9]:
print(params[0].size())

torch.Size([6, 1, 5, 5])


In [10]:
print(params[1].size())

torch.Size([6])


In [11]:
print(params[2].size())

torch.Size([16, 6, 5, 5])


In [12]:
print(params[3].size())

torch.Size([16])


In [13]:
print(params)

[Parameter containing:
(0 ,0 ,.,.) = 
  0.1609  0.1418 -0.1929  0.1251 -0.0802
  0.1654 -0.0955  0.1523 -0.1061  0.0423
 -0.1432 -0.1374  0.1380  0.1395  0.0986
  0.0458  0.1220  0.0859 -0.0048  0.0316
 -0.1140 -0.0974 -0.0903 -0.0556 -0.0511

(1 ,0 ,.,.) = 
  0.1725 -0.1697 -0.0894  0.1813  0.1547
  0.1855  0.0730  0.0703 -0.0323  0.0834
  0.0150  0.0016 -0.1991  0.1197  0.0476
  0.0159 -0.0141 -0.1757 -0.1675 -0.1524
 -0.0825  0.1435 -0.0673 -0.1666 -0.1215

(2 ,0 ,.,.) = 
  0.1541  0.1610 -0.1984 -0.1554 -0.0720
  0.1876  0.0226 -0.0735  0.0470 -0.1477
 -0.1549  0.0237 -0.1148 -0.0905 -0.0915
  0.0154 -0.0012  0.1601  0.1924 -0.0985
  0.1723  0.1463  0.0097  0.0626 -0.1366

(3 ,0 ,.,.) = 
  0.0739  0.1847 -0.1897 -0.0956 -0.1144
 -0.0122  0.1020  0.0556 -0.0621  0.1804
 -0.1098  0.1886  0.0845 -0.1349 -0.0125
 -0.0404 -0.1927  0.0899 -0.0285 -0.0791
 -0.0825 -0.1415  0.0044  0.0460  0.1105

(4 ,0 ,.,.) = 
  0.0897  0.0964  0.1764 -0.1775  0.0839
  0.1792 -0.0937 -0.1957 -0.1764 -0.1

In [14]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(out)

Variable containing:
1.00000e-02 *
 -7.1975 -3.9318 -2.6172 -0.5254 -6.7064 -3.9373  4.1867 -4.1266  5.4283  8.3051
[torch.FloatTensor of size 1x10]



In [15]:
net.zero_grad()

In [16]:
out.backward(torch.randn(1, 10))

In [17]:
output = net(input)

In [18]:
target = Variable(torch.arange(1, 11))
criterion = nn.MSELoss()

loss = criterion(output, target)

In [19]:
print(loss)

Variable containing:
 38.4106
[torch.FloatTensor of size 1]



In [20]:
print(loss.grad_fn)

In [22]:
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [23]:
net.zero_grad()

In [24]:
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]



In [25]:
loss.backward()

In [26]:
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad after backward
Variable containing:
 0.0275
-0.0402
 0.0590
 0.0460
-0.0441
-0.1042
[torch.FloatTensor of size 6]



In [28]:
# weight = weight - learning_rate * gradient
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [29]:
import torch.optim as optim

In [30]:
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [31]:
optimizer.zero_grad()

In [32]:
output = net(input)

In [33]:
loss = criterion(output, target)


In [34]:
loss.backward()

In [35]:
optimizer.step()